# **UC3. How many subscribers will be on-line in my network next week?**

For several reasons, we need to predict what the network will look like at a certain point in the future. We want to forecast traffic accurately to plan capacity provisioning in advance, forecast subscribers to plan for IP address pool capacity, and forecast user demand to plan for our K8s cluster capacity. Failure to do so could result in service unavailability, missed SLAs, and potential customer churn.

So, how can we make reasonable predictions for a specific metric to take action now based on future forecasts? This is the main issue we aim to solve. In this case, we want to properly plan the capacity of IP address pools for our BNGs by estimating the number of subscribers expected over the coming days at different hours. If successful, this will provide a solid basis for determining the necessary size of the IP pools and provisioning them accordingly.

# Connection to drive and path definition (Just for Google Colab Lab)




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
path_files ='/content/drive/MyDrive/Colab Notebooks/Files'

# ***OR***
# Connection path definition (Just for AWS Jupiter Notebook)

In [ ]:
import sys
path_files ='./Files'

# Import libraries

In [ ]:
pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
import time
from calendar import timegm, monthrange
from datetime import datetime, timedelta
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from os import listdir
from os.path import isfile, join

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.api import ExponentialSmoothing
from scipy.stats import iqr

# Data Collection

We will load a data set that contains number of subscribers for PE-5 for a period of 29 days, with 5 minute frequency

In [ ]:
metric_df=pd.read_csv(join(path_files,'bng_subscribers_metric_forecast.csv'),index_col=0)

In [ ]:
metric_df

# Data preparation

In [ ]:
ds_column = []
for i,obs in metric_df.iterrows():
    #new_timestamp = datetime.fromtimestamp(obs['timestamp']).strftime('%Y-%m-%d %H:%M:%S')
    new_timestamp = datetime.fromtimestamp(obs['timestamp'])
    ds_column.append(new_timestamp)
metric_df['ds']=ds_column
metric_df['y']=metric_df['bng_subscribers']



In [ ]:
metric_df

In [ ]:
# Visualize data using seaborn
sns.set(rc={'figure.figsize':(12,8)})
sns.lineplot(x=metric_df['ds'], y=metric_df['y'])
plt.title('BNG Subscribers')

In [ ]:
df = metric_df[['ds','y']].set_index('ds')


In [ ]:
def seasonal_decompose (df):
    decomposition = sm.tsa.seasonal_decompose(df, model='additive',period=2016)

    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    fig = decomposition.plot()
    fig.set_size_inches(14, 7)
    plt.show()

    return trend, seasonal, residual

seasonal_decompose(df)

In statsmodels, when you use *decomposition.resid*, it gives you the residual component of the time series after removing the trend and seasonality. The residual represents the part of the data that cannot be explained by the trend and seasonality, and it typically includes noise or random fluctuations.

# Naive model

We want to forecast 1 day, which means 288 observations.

**Approach 1:** The "naïve approach" in time series forecasting is a simple method where the forecast for the next time point is assumed to be exactly the same as the previous observed value.
In other words, for each new time period, the model predicts that the value will remain unchanged from the last observed value. This approach works under the assumption that there is no significant trend, seasonality, or pattern influencing future values. It is often used as a baseline model to compare the performance of more sophisticated forecasting methods.


To do this, We will use the data from an entire seasonal period, which is one week (2016 observations) >>>

naive prediction: y_pred = y[t-2016]

In [ ]:
y_pred_naive = df['y'][:-2016].values.reshape(-1,1)

In [ ]:
y_pred_naive.shape

In [ ]:
df.loc[df.index[2016:],'y_pred_naive']=y_pred_naive

In [ ]:
df

We can estimate how well the model is performing by comparing the predicted value with the actual value of the metric at each point in time. We will use two metrics: **Mean Absolute Error** (MAE), which estimates the error based on the absolute difference. However, absolute error may not always be suitable as it does not provide a relative perspective based on the actual values. Alternatively, we have **Mean Absolute Percentage Error** (MAPE), which normalizes the error relative to the actual values, offering a better sense of the degree or proportion of deviations.

In [ ]:
performance_naive_MAE = mean_absolute_error(df['y'][2016:], df['y_pred_naive'][2016:])
print(f'The MAE for the model is {performance_naive_MAE}')

performance_naive_MAPE = mean_absolute_percentage_error(df['y'][2016:], df['y_pred_naive'][2016:])
print(f'The MAPE for the model is {performance_naive_MAPE}')

In [ ]:
sns.lineplot(x=df.index[7000:], y='y', data=df[7000:], color='black')
sns.lineplot(x=df.index[8000:8300], y='y_pred_naive', data=df[8000:8300], color='red')

# Holt Winters

**Approach 2:** The **Holt-Winters** technique, also known as **triple exponential smoothing**, is a forecasting method used for time series data that includes three main components:

1. **Level (L)**: The estimate of the average or baseline value of the time series.
2. **Trend (T)**: The rate of change or direction in which the time series is moving (upward or downward).
3. **Seasonality (S)**: The repeating fluctuations occurring at regular intervals, such as daily, monthly, or yearly patterns.

This model adjusts these three components over time, allowing for more accurate forecasts, especially in time series with seasonality and trends.

The Holt-Winters model has two main versions:
- **Additive**: Used when the magnitude of seasonality is constant over time.
- **Multiplicative**: Used when the magnitude of seasonality changes in proportion to the trend.

The general Holt-Winters formula involves three equations that update the level, trend, and seasonality at each time step, adjusting the forecasts according to these components.

In [ ]:
def holtwin(y_to_train, seasonal_period, predict_date):

    holt = ExponentialSmoothing(y_to_train, seasonal_periods=seasonal_period, trend='add', seasonal='add',use_boxcox=True).fit()
    y_forecast = holt.forecast(predict_date).rename('Additive')
    return y_forecast


We used the observations up to position 8000 to train the model and requested a prediction for the next 300 observations (just over one day). We also informed the model of the identified seasonality, which in this case is seven days (2016 observations).

**Warning: Next task will last around 5 minutes**

In [ ]:
y_holtwin_pred = holtwin(df['y'][:8000],2016,300)

In [ ]:
performance_holtwint_MAE = mean_absolute_error(df['y'][8000:8300], y_holtwin_pred)
print(f'The MAE for the model is {performance_holtwint_MAE}')
performance_holtwint_MAPE = mean_absolute_percentage_error(df['y'][8000:8300], y_holtwin_pred)
print(f'The MAPE for the model is {performance_holtwint_MAPE}')

In [ ]:
sns.lineplot(x=df.index[7000:], y='y', data=df[7000:], color='black')
sns.lineplot(x=df.index[8000:8300], y=y_holtwin_pred, color='red')

We can still observe a quite noisy prediction, likely due to the fact that the anomalies are causing the model to produce this variability in the forecast as well.